# Import Dataset

In [2]:
import kagglehub
import pandas as pd

SEED = 42

# Download latest version
path = kagglehub.dataset_download("austinreese/craigslist-carstrucks-data")

print("Path to dataset file:", path)

# Load the dataset
csv_file = f"{path}/vehicles.csv"
raw_data = pd.read_csv(csv_file)

# Print columns
print("Columns:", raw_data.columns)

C:\Users\guilh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset file: C:\Users\guilh\.cache\kagglehub\datasets\austinreese\craigslist-carstrucks-data\versions\10
Columns: Index(['id', 'url', 'region', 'region_url', 'price', 'year', 'manufacturer',
       'model', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status',
       'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color',
       'image_url', 'description', 'county', 'state', 'lat', 'long',
       'posting_date'],
      dtype='object')


# Data Cleaning

## Common stuff

In [3]:
cars_columns = ['year', 'manufacturer', 'model', 'condition', 'cylinders', 'fuel', 
                    'odometer', 'transmission', 'VIN', 'drive', 'size', 'type', 'paint_color']

listings_columns = ['description', 'posting_date', 'price']

columns_to_keep = cars_columns + listings_columns

filtered_data = raw_data[columns_to_keep].copy()

# fill missing values with mode
filtered_data = filtered_data.fillna(filtered_data.mode().iloc[0])

# drop duplicate VINs
filtered_data = filtered_data.drop_duplicates(subset=['VIN'])

# Add new column carId with consecutive numbers from 1
filtered_data['carId'] = range(1, len(filtered_data) + 1)

print("Total: ", len(filtered_data))
print(filtered_data.head())

Total:  118264
      year manufacturer                     model condition    cylinders fuel  \
0   2017.0         ford                     f-150      good  6 cylinders  gas   
27  2014.0          gmc  sierra 1500 crew cab slt      good  8 cylinders  gas   
28  2010.0    chevrolet            silverado 1500      good  8 cylinders  gas   
29  2020.0    chevrolet       silverado 1500 crew      good  8 cylinders  gas   
30  2017.0       toyota      tundra double cab sr      good  8 cylinders  gas   

    odometer transmission                VIN drive       size    type  \
0   100000.0    automatic  1FMJU1JT1HEA52352   4wd  full-size   sedan   
27   57923.0        other  3GTP1VEC4EG551563   4wd  full-size  pickup   
28   71229.0        other  1GCSCSE06AZ123805   4wd  full-size  pickup   
29   19160.0        other  3GCPWCED5LG130317   4wd  full-size  pickup   
30   41124.0        other  5TFRM5F17HX120972   4wd  full-size  pickup   

   paint_color                                        descr

## Cars DB

In [4]:
# Filter the data to include only car columns
cars_data = filtered_data[['carId'] + cars_columns].copy()

# Rename columns to match the SQL format
cars_data = cars_data.rename(columns={
    'carId': 'car_id',
    'year': 'car_year',
    'manufacturer': 'car_manufacturer',
    'model': 'car_model',
    'condition': 'car_condition',
    'cylinders': 'car_cylinders',
    'fuel': 'car_fuel',
    'odometer': 'car_odometer',
    'transmission': 'car_transmission',
    'VIN': 'car_vin',
    'drive': 'car_drive',
    'size': 'car_size',
    'type': 'car_type',
    'paint_color': 'car_paint_color'
})

# Define column limits based on SQL schema
column_limits = {
    'car_manufacturer': 100,
    'car_model': 100,
    'car_condition': 50,
    'car_cylinders': 50,
    'car_fuel': 50,
    'car_transmission': 50,
    'car_vin': 17,
    'car_drive': 50,
    'car_size': 50,
    'car_type': 50,
    'car_paint_color': 50
}

# Apply truncation to string columns
for col, limit in column_limits.items():
    cars_data[col] = cars_data[col].astype(str).str.slice(0, limit)

# Ensure car_year and car_odometer are integers
cars_data['car_year'] = cars_data['car_year'].fillna(0).astype(int)
cars_data['car_odometer'] = cars_data['car_odometer'].fillna(0).astype(int)

# Save the filtered data to a new CSV file
cars_data.to_csv("cars.csv", index=False)

print("Number of cars:", len(cars_data))
print(cars_data.head())


Number of cars: 118264
    car_id  car_year car_manufacturer                 car_model car_condition  \
0        1      2017             ford                     f-150          good   
27       2      2014              gmc  sierra 1500 crew cab slt          good   
28       3      2010        chevrolet            silverado 1500          good   
29       4      2020        chevrolet       silverado 1500 crew          good   
30       5      2017           toyota      tundra double cab sr          good   

   car_cylinders car_fuel  car_odometer car_transmission            car_vin  \
0    6 cylinders      gas        100000        automatic  1FMJU1JT1HEA52352   
27   8 cylinders      gas         57923            other  3GTP1VEC4EG551563   
28   8 cylinders      gas         71229            other  1GCSCSE06AZ123805   
29   8 cylinders      gas         19160            other  3GCPWCED5LG130317   
30   8 cylinders      gas         41124            other  5TFRM5F17HX120972   

   car_drive   

## CarListings DB

In [5]:
# Filter the data to include only listings columns
listings_data = filtered_data[['carId'] + listings_columns].copy()

# Keep only the first half listings
listings_data = listings_data.head(len(listings_data) // 2)

# Rename price column to sale_price
listings_data = listings_data.rename(columns={'price': 'sale_price'})

# Add new column listingId with consecutive numbers from 1
listings_data['listingId'] = range(1, len(listings_data) + 1)

# Add new column userId all same value 1
listings_data['userId'] = 1

# Add new column type with 20% of the listings being "rent" and the rest "buy"
listings_data['type'] = 'TypeEnum_BUY'
listings_data.loc[listings_data.sample(frac=0.2, random_state=SEED).index, 'type'] = 'TypeEnum_RENT'

# Add new column promoted with 10% of the listings being True and the rest False
listings_data['promoted'] = False
listings_data.loc[listings_data.sample(frac=0.1, random_state=SEED).index, 'promoted'] = True

# Add new column status
listings_data['status'] = 'StatusEnum_AVAILABLE'

# Randomly select 50% of "buy" listings and set their status to "SOLD"
buy_indices = listings_data[listings_data['type'] == 'TypeEnum_BUY'].sample(frac=0.5, random_state=SEED).index
listings_data.loc[buy_indices, 'status'] = 'StatusEnum_SOLD'

# Randomly select 25% of "rent" listings and set their status to "RENTED"
rent_indices = listings_data[listings_data['type'] == 'TypeEnum_RENT'].sample(frac=0.25, random_state=SEED).index
listings_data.loc[rent_indices, 'status'] = 'StatusEnum_RESERVED'

# Reorder the columns
listings_data = listings_data[['listingId', 'carId', 'userId', 'type', 'description', 'posting_date', 'sale_price', 'promoted', 'status']]

# Clean description column and replace it with "Great car" for the first half of the listings and "Bad car" for the second half
listings_data['description'] = ['Great car' if i < len(listings_data) / 2 else 'Bad car' for i in range(len(listings_data))]

listings_data = listings_data.rename(columns={
    'listingId': 'listing_id',
    'carId': 'listing_car_id',
    'userId': 'listing_user_id',
    'type': 'listing_type',
    'description': 'listing_description',
    'posting_date': 'listing_posting_date',
    'sale_price': 'listing_sale_price',
    'promoted': 'listing_promoted',
    'status': 'listing_status'
})

# Save the filtered data to a new CSV file
listings_data.to_csv("listings.csv", index=False)

print("Number of listings:", len(listings_data))
print(listings_data.head())

Number of listings: 59132
    listing_id  listing_car_id  listing_user_id   listing_type  \
0            1               1                1   TypeEnum_BUY   
27           2               2                1   TypeEnum_BUY   
28           3               3                1   TypeEnum_BUY   
29           4               4                1   TypeEnum_BUY   
30           5               5                1  TypeEnum_RENT   

   listing_description      listing_posting_date  listing_sale_price  \
0            Great car  2021-04-23T22:13:05-0400                6000   
27           Great car  2021-05-04T12:31:18-0500               33590   
28           Great car  2021-05-04T12:31:08-0500               22590   
29           Great car  2021-05-04T12:31:25-0500               39590   
30           Great car  2021-05-04T10:41:31-0500               30990   

    listing_promoted        listing_status  
0              False       StatusEnum_SOLD  
27             False       StatusEnum_SOLD  
28       